## NYC Neighborhood Desirability/Restaurants: 

# Buying a Home in NYC: What Neighborhoods are the Best Value?
### Applying Data Science Tools to Understand NYC's Residential Real Estate Fundamentals

    Josh Grasso | joshgrasso@gmail.com
This project seeks to understand the fundamental factors that explain differences in residential real estate prices across NYC. 

### Neighborhood Desirability: via FourSquare Restaurant Ratings

Williamsburg, Brooklyn is not a convenient starting point to commute to work in Manhattan.  But the neighborhood is fun. The same debate holds for living downtown – yes, it’s further from your office in Midtown, but the neighborhoods are so much more livable. They have more amenities, more trendy restaurants, more culture. To try to capture the desirability of living in a neighborhood, I turned to the visitor/user ratings of local restaurants on FourSquare.  

The FourSquare Rating of a venue/restaurant ranges from 0.0 to 10.0, with a rating above 9.0 being few and far between – especially outside of a major city. Ratings above 8.0 are solid, and fast-food favorites like Chipotle or Five Guys are often in above 7.0 – just to give some sense of scale. FourSquare’s rating are global and relevant; Postmates’ food deliver service is powered by the FourSquare restaurant database. In the main Jupyter Notebook of this project I run a regression of three ratings criteria against housing prices, to see which series holds more information about the relative difference in prices across neighborhoods: all restaurants rated above 8.0, above 8.5, and above 9.0. Ratings above 8.5 had a stronger r-squared than the other two, and proved to be as-strong/stronger than commute time in understanding home prices. 

The search is broken up into two parts, one to get all venues across NYC neighborhoods, and the second to get the venue details – the Rating. FourSquare limits the volume of venue detail requests that can be made in a day to 500. So with nearly 300 neighborhoods x 50 restaurants/neighborhood – it took two week to run the second half of the analysis. 

The first part of the search, asks for all venues that fall under the two “Categories:” 'Food' and 'Food & Drink Shop' that are within a radius of 0.5 miles, roughly a 10min walk, from the center of each neighborhood. FourSquare allows a maximum of 50 venues/restaurants to be returned for each venue search – so I had to optimize the search radius for Manhattan – where neighborhoods have a smaller geographic footprint and lots of great restaurants, and outer boroughs were neighborhoods have a larger geographic footprint and sparser restaurant choices. Finally, the search is optimized for relevance, with the search-time being Saturday at 5pm – a favorite time to go out.  


In [1]:
import numpy as np
import pandas as pd
import requests

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
#plt.style.use('seaborn')  
sns.set()

from datetime import datetime
today = datetime.now()
month,day,year = today.month,today.day,today.year

In [2]:
from pathlib import Path
home_path = Path.home() / 'Jupyter' / 'Real_Estate'

In [4]:
# Neighborhood Coordinates: 
# Neighborhoods Names
# https://www1.nyc.gov/site/planning/data-maps/open-data.page

neighborhood_url = 'https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/Neighborhood_Names/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson'
resp = requests.get(neighborhood_url)
neighborhood_json = resp.json()

neighborhood_ids_list = []
neighborhood_details_list = []

for neighborhood_dict in neighborhood_json['features']:
    neighborhood_ids_list.append(neighborhood_dict['id']) 
    
    d = {}
    d['ID'] = neighborhood_dict['id']
    d['Name'] = neighborhood_dict['properties']['Name']
    d['Borough'] = neighborhood_dict['properties']['Borough']
    d['Lat'] = neighborhood_dict['geometry']['coordinates'][1]
    d['Long'] = neighborhood_dict['geometry']['coordinates'][0]
    
    neighborhood_details_list.append(d)

neighborhood_df = pd.DataFrame.from_dict(neighborhood_details_list)
neighborhood_df['Borough'] = neighborhood_df['Borough'].replace({'Staten Island': 'Staten_Island'})
neighborhood_df = neighborhood_df.set_index(['Borough', 'Name'])

In [5]:
neighborhood_df

ID        Lat       Long
Borough       Name                                    
Bronx         Wakefield        1  40.894713 -73.847202
              Co-op City       2  40.874302 -73.829941
              Eastchester      3  40.887564 -73.827808
              Fieldston        4  40.895446 -73.905644
              Riverdale        5  40.890843 -73.912587
...                          ...        ...        ...
Staten_Island Stapleton      302  40.626936 -74.077903
              Rosebank       303  40.615313 -74.069807
              West Brighton  304  40.631887 -74.107183
              Grymes Hill    305  40.624193 -74.087250
              Todt Hill      306  40.597077 -74.111330

[306 rows x 3 columns]

In [6]:
# Resource:
# https://developer.foursquare.com/docs/places-api/
# https://github.com/hongyuaw/Coursera_Capstone/blob/Master/Capstone_project_(week_5).ipynb

In [7]:
# Call My FourSquare API Key:  
# 'My_FourSquare_Client_Secret.txt'
# 'My_FourSquare_Client_ID.txt'

FourSquare_Client_Secret_path = Path.home() / 'Jupyter' / 'My_FourSquare_Client_Secret.txt'
FourSquare_Client_ID_path = Path.home() / 'Jupyter' / 'My_FourSquare_Client_ID.txt'

if FourSquare_Client_Secret_path.is_file():
    FourSquare_Client_Secret_file = open(FourSquare_Client_Secret_path, 'rt')
    My_FourSquare_Client_Secret = FourSquare_Client_Secret_file.read()
    FourSquare_Client_Secret_file.close()
else:
    print('Error: FourSquare_Client_Secret File not found')

if FourSquare_Client_ID_path.is_file():
    FourSquare_Client_ID_file = open(FourSquare_Client_ID_path, 'rt')
    My_FourSquare_Client_ID = FourSquare_Client_ID_file.read()
    FourSquare_Client_ID_file.close()
else:
    print('Error: FourSquare_Client_ID File not found')

# My_FourSquare_Client_Secret
# My_FourSquare_Client_ID
Version = '20210310'

My_FourSquare_dict = {'Client_Secret': My_FourSquare_Client_Secret,
                     'Client_ID': My_FourSquare_Client_ID,
                     'Version': '20210310'}


In [8]:
# FourSquare Place Categories: 
categories_url = ('https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'
                  .format(My_FourSquare_Client_ID, My_FourSquare_Client_Secret, Version))
categories_json = requests.get(categories_url).json()

# Query Variables:
# categories_json['response']['categories']

# Categories:
# 'Food' '4d4b7105d754a06374d81259'
# 'Food & Drink Shop' '4bf58dd8d48988d1f9941735'

In [165]:
def foursquare_get_venues(My_FourSquare_dict, lat, long, radius_miles=0.5, limit=50, 
                             categoryId=['4d4b7105d754a06374d81259', '4bf58dd8d48988d1f9941735'],
                             day='Saturday', sortByPopularity=0, time='17:00'):
    # 'https://maps.googleapis.com/maps/api/directions/json?parameters'
    
    # https://developer.foursquare.com/docs/api-reference/venues/explore/
    radius_meters = radius_miles * 1610
    
    # Build URL:
    gmaps_url = 'https://maps.googleapis.com/maps/api/directions/json?'
    gmaps_url += "&origin={},{}".format(lat, long)
    
    gmaps_url += "&origin={}".format(My_FourSquare_dict['Client_ID'])
    venues_url += "&client_secret={}".format(My_FourSquare_dict['Client_Secret'])
    venues_url += "&v={}".format(My_FourSquare_dict['Version'])
    venues_url += "&ll={},{}".format(lat, long)
    venues_url += "&radius={}".format(radius_meters)
    venues_url += "&limit={}".format(limit)
    venues_url += "&day={}".format(day)
    venues_url += "&time={}".format(time)
    venues_url += "&sortByPopularity={}".format(sortByPopularity)
    # sortByPopularity: [1]:  Boolean flag to sort the results by popularity instead of relevance
    
    categories_string = ','.join(test_categories)
    venues_url += "&categoryId={}".format(categories_string)
    # venues_url += "&query=dinner"

    return requests.get(venues_url).json()

In [169]:
def parse_venues(foursquare_venues_json):
    # Initialize Venue Entries
    key_list = ['ID', 'Venue_Name', 'Venue_Type', 'Lat', 'Long', 'Distance', 'FourSquare_Neighborhood'
                'Address', 'Cross_Street', 'Postal_Code', 'City', 'State']
    # Venues:
    venue_list = []
    for i, venue in enumerate(foursquare_venues_json['response']['groups'][0]['items']):
        # Recall: venue = test_park_slope_json['response']['groups'][0]['items'][i]
        # Reset to next venue:
        venue_dict = {}
        for k in key_list: 
            venue_dict[k] = None
        
        try:
            venue_dict['ID'] = venue['venue']['id']
        except (IndexError, ValueError, KeyError):
            venue_dict['ID'] = np.nan
        try:
            venue_dict['Venue_Name'] = venue['venue']['name']
        except (IndexError, ValueError, KeyError):
            venue_dict['Venue_Name'] = np.nan    
        try:
            venue_dict['Lat'] = venue['venue']['location']['lat']
        except (IndexError, ValueError, KeyError):
            venue_dict['Lat'] = np.nan
        try:
            venue_dict['Long'] = venue['venue']['location']['lng']
        except (IndexError, ValueError, KeyError):
            venue_dict['Long'] = np.nan
        try:
            venue_dict['Distance'] = round(float(venue['venue']['location']['distance']) / 1610, 2)
        except (IndexError, ValueError, KeyError):
            venue_dict['Distance'] = np.nan
        try:
            venue_dict['FourSquare_Neighborhood'] = foursquare_venues_json['response']['headerLocation']
        except (IndexError, ValueError, KeyError):
            venue_dict['Distance'] = np.nan
        try:
            venue_dict['Address'] = venue['venue']['location']['address']
        except (IndexError, ValueError, KeyError):
            venue_dict['Address'] = np.nan    
        try:
            venue_dict['Cross_Street'] = venue['venue']['location']['crossStreet']
        except (IndexError, ValueError, KeyError):
            venue_dict['Cross_Street'] = np.nan
        try:
            venue_dict['Postal_Code'] = venue['venue']['location']['postalCode']
        except (IndexError, ValueError, KeyError):
            venue_dict['Postal_Code'] = np.nan
        try:
            venue_dict['City'] = venue['venue']['location']['city']
        except (IndexError, ValueError, KeyError):
            venue_dict['City'] = np.nan
        try:
            venue_dict['State'] = venue['venue']['location']['state']
        except (IndexError, ValueError, KeyError):
            venue_dict['State'] = np.nan  
        
        for category in venue['venue']['categories']: #[0]['name']
            if category['primary'] == True:
                try:
                    venue_dict['Venue_Type'] = category['name']
                except (IndexError, ValueError, KeyError):
                    venue_dict['Venue_Type'] = np.nan
        venue_list.append(venue_dict)
    return pd.DataFrame.from_dict(venue_list)

In [ ]:
# Step 1: Search for Local Restaurants

In [265]:
# Split into Two Loops: Get Venues and Venue Details

# Loop 1: Get Venues & Save to CSV

all_restaurants_df = pd.DataFrame()
for x, y in zip(neighborhood_df.reset_index(drop=False)['Borough'],
                neighborhood_df.reset_index(drop=False)['Name']):
    #print(neighborhood_df['Lat'].loc[x, y])
    #print(neighborhood_df['Long'].loc[x, y])
    _json = foursquare_get_venues(My_FourSquare_dict, 
                                  lat=neighborhood_df['Lat'].loc[x, y], 
                                  long=neighborhood_df['Long'].loc[x, y])
    _df = parse_venues(_json)
    _df['Borough'] = x
    _df['NYC_Neighborhood'] = y
    all_restaurants_df = all_restaurants_df.append(_df)


In [292]:
all_restaurants_df

,ID,Venue_Name,Venue_Type,Lat,Long,Distance,Address,Cross_Street,Postal_Code,City,State,NYC_Neighborhood,Borough
0,4d375ce799fe8eec99fd2355,Ripe Kitchen & Bar,Caribbean Restaurant,40.898152,-73.838875,0.50,151 W Sandford Blvd,South 11th Avenue,10550,Mount Vernon,NY,Wakefield,Bronx
1,4c10f6aece57c92804a682d2,Jackie's West Indian Bakery,Caribbean Restaurant,40.889283,-73.843310,0.43,1203 E 233rd St,Baychester Aveue,10466,Bronx,NY,Wakefield,Bronx
2,4d33665fb6093704b80001e0,Subway,Sandwich Place,NaN,-73.849152,0.31,980 E 233rd St,NaN,10466,Bronx,NY,Wakefield,Bronx
3,4c25c212f1272d7f836385c5,Dunkin',Donut Shop,40.890459,-73.849089,0.31,980 E 233rd St,Paulding Ave,10466,Bronx,NY,Wakefield,Bronx
4,508af256e4b0578944c87392,Cooler Runnings Jamaican Restaurant,Caribbean Restaurant,40.898083,-73.850259,0.28,1802 Nereid Ave,Wickham,10466,Bronx,NY,Wakefield,Bronx
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,4e28cdc8e4cd6c6cb34b2322,NaN,Pizza Place,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New York,Grymes Hill,Staten_Island
6,4f32043a19833175d60a05a4,Chieng Garden Kitchen,Chinese Restaurant,NaN,-74.080989,0.33,183 Broad St,NaN,10304,Staten Island,NY,Grymes Hill,Staten_Island
7,4e1f41f3a809eb325ff7dc82,NaN,Pizza Place,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New York,Grymes Hill,Staten_Island
8,4f68c264e4b06cd27a40957a,Romeo's Pizza,Pizza Place,40.624838,-74.079379,0.41,117 Broad St,Canal St,10304,Staten Island,NY,Grymes Hill,Staten_Island


In [ ]:
# Duplicate restaurants?  # 1,945
# all_restaurants_df[all_restaurants_df['ID'].duplicated(keep='first')].sort_values('ID', ascending=True)

In [ ]:
# Save to CSV:
all_restaurants_df.to_csv(path_or_buf= home_path / 'FourSquare_All_Restaurants.csv')


In [9]:
# Step 2: Get Ratings for each Restaurant

In [10]:
def foursquare_venue_details(My_FourSquare_dict, venue_id):
    # Build URL:
    venues_url = 'https://api.foursquare.com/v2/venues/{}?'.format(venue_id)
    venues_url += "&client_id={}".format(My_FourSquare_dict['Client_ID'])
    venues_url += "&client_secret={}".format(My_FourSquare_dict['Client_Secret'])
    venues_url += "&v={}".format(My_FourSquare_dict['Version'])
    return requests.get(venues_url).json()

In [11]:
def parse_venue_details(foursquare_venue_details_json):
    # Initialize Venue Entries
    key_list = ['ID', 'Venue_Name', 'Rating', 
                'Price',  # price tier from 1 (least pricey) - 4 (most pricey)
                'Created_At',  # createdAt: Seconds since epoch when the venue was created.
                'Checkins', 'Users', 'Tips', 'Visits'] # Stats:
    venue_dict = {}
    for k in key_list: 
        venue_dict[k] = None
        
    try:
        venue_dict['ID'] = foursquare_venue_details_json['response']['venue']['id']
    except (IndexError, ValueError, KeyError):
        venue_dict['ID'] = np.nan
    try:
        venue_dict['Venue_Name'] = foursquare_venue_details_json['response']['venue']['name']
    except (IndexError, ValueError, KeyError):
        venue_dict['Venue_Name'] = np.nan    
    try:
        venue_dict['Rating'] = foursquare_venue_details_json['response']['venue']['rating']
    except (IndexError, ValueError, KeyError):
        venue_dict['Rating'] = np.nan
    try:
        venue_dict['Price'] = foursquare_venue_details_json['response']['venue']['price']['tier']
    except (IndexError, ValueError, KeyError):
        venue_dict['Price'] = np.nan
    try:
        venue_dict['Created_At'] = foursquare_venue_details_json['response']['venue']['createdAt']
    except (IndexError, ValueError, KeyError):
        venue_dict['Created_At'] = np.nan
    
    try:
        venue_dict['Checkins'] = foursquare_venue_details_json['response']['venue']['stats']['checkinsCount']
    except (IndexError, ValueError, KeyError):
        venue_dict['Checkins'] = np.nan
    try:
        venue_dict['Users'] = foursquare_venue_details_json['response']['venue']['stats']['usersCount']
    except (IndexError, ValueError, KeyError):
        venue_dict['Users'] = np.nan
    try:
        venue_dict['Tips'] = foursquare_venue_details_json['response']['venue']['stats']['tipCount']
    except (IndexError, ValueError, KeyError):
        venue_dict['Tips'] = np.nan
    try:
        venue_dict['Visits'] = foursquare_venue_details_json['response']['venue']['stats']['visitsCount']
    except (IndexError, ValueError, KeyError):
        venue_dict['Visits'] = np.nan  
    
    return pd.Series(venue_dict) #venue_dict #pd.DataFrame.from_dict(venue_dict, )


In [12]:
# Loop 2: Get Rating and Details for each (unique) Venue

In [13]:
all_details_df = pd.read_csv(filepath_or_buffer= home_path / 'FourSquare_Details_All_Restaurants.csv',
                             index_col=0)
all_details_df

,ID,Address,Borough,Checkins,City,Created_At,Cross_Street,Distance,Lat,Long,NYC_Neighborhood,Postal_Code,Price,Rating,State,Tips,Users,Venue_Name,Venue_Type,Visits
0,3fd66200f964a52006e91ee3,1274 Amsterdam Ave,Manhattan,NaN,New York,1.071014e+09,W 123rd St.,0.40,40.811241,-73.957869,Manhattanville,10027,2.0,7.8,NY,60.0,NaN,Max SoHa,Italian Restaurant,NaN
1,3fd66200f964a52044ea1ee3,1274 Amsterdam Ave,Manhattan,NaN,New York,1.071014e+09,W 123rd St.,0.39,40.811241,-73.957869,Morningside Heights,10027,2.0,8.7,NY,470.0,NaN,J.G. Melon,Italian Restaurant,NaN
2,3fd66200f964a5205fee1ee3,72 Bedford St,Manhattan,NaN,New York,1.071014e+09,Commerce,0.22,40.731493,-74.004825,West Village,10014,3.0,7.7,NY,17.0,NaN,Ponte Vecchio Restaurant,Brazilian Restaurant,NaN
3,3fd66200f964a5207bf11ee3,71 Irving Pl,Manhattan,NaN,New York,1.071014e+09,btwn E 18th & E 19th St,0.29,40.736716,-73.986817,Gramercy,10003,3.0,8.6,NY,72.0,NaN,F & J Pine Tavern,Café,NaN
4,3fd66200f964a52080e41ee3,172 Thompson St,Manhattan,NaN,New York,1.071014e+09,btwn Bleecker & Houston St,0.06,40.727743,-74.000267,Greenwich Village,10012,4.0,8.9,NY,122.0,NaN,Craft,Sushi Restaurant,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10420,604ae77621921d08a37d8cd4,825 Morris Park Ave,Bronx,NaN,New York,NaN,Matthews Ave,0.31,40.846332,-73.861653,Van Nest,10462,NaN,NaN,NY,NaN,NaN,La Catrina,Mexican Restaurant,NaN
10421,604bebdb59c1ce087b17a4a5,1761 Crosby Ave,Bronx,NaN,Bronx,NaN,NaN,0.45,NaN,-73.832030,Country Club,10461,NaN,NaN,NY,NaN,NaN,Taqueria Guerrero,Mexican Restaurant,NaN
10422,604bebdb59c1ce087b17a4a5,1761 Crosby Ave,Bronx,NaN,Bronx,NaN,NaN,0.28,NaN,-73.832030,Pelham Bay,10461,NaN,NaN,NY,NaN,NaN,Taqueria Guerrero,Mexican Restaurant,NaN
10423,604e11201ee66f712571074f,259-19 Union Tpke,Queens,NaN,Glen Oaks,NaN,NaN,0.23,NaN,-73.712715,Glen Oaks,11004,NaN,NaN,NY,NaN,NaN,Subway Sandwiches,Sandwich Place,NaN


In [14]:
all_details_df['ID'].isna().sum()

0

In [15]:
# Develop a list of new, unique ID's to search:
# FourSquare caps the free, dailiy volume to 500 restaurant-details searches 

# Search only unique ID's: some restaurants are in mutiple neighborhoods
new_details_search_df = all_details_df.drop_duplicates('ID', keep='first').sort_values('ID', ascending=True)

# Remove restaurants that have ratings already
new_details_search_df = new_details_search_df[new_details_search_df['Rating'].isna()]


In [16]:
new_details_search_df

,ID,Address,Borough,Checkins,City,Created_At,Cross_Street,Distance,Lat,Long,NYC_Neighborhood,Postal_Code,Price,Rating,State,Tips,Users,Venue_Name,Venue_Type,Visits
6425,4f32495719836c91c7c8bf82,4374 White Plains Rd,Bronx,NaN,Bronx,NaN,NaN,0.44,NaN,-73.854638,Wakefield,10466,NaN,NaN,NY,NaN,NaN,Him Health Food Market,Food,NaN
6560,4f36bdd1e4b08533d18f451a,NaN,Bronx,NaN,Bronx,NaN,NaN,0.49,NaN,-73.924668,Concourse Village,10451,NaN,NaN,NY,NaN,NaN,NaN,Pizza Place,NaN
8972,57b544bd498e614030ff5cf2,"2255 Emmons Ave,11235",Brooklyn,NaN,Brooklyn,NaN,NaN,0.22,NaN,-73.944520,Sheepshead Bay,11235,NaN,NaN,NY,NaN,NaN,Opera Cafe Lunch,Food Court,NaN
9815,5b37adcf90d1ed00391e6fa8,8614 5th Avenue,Brooklyn,NaN,New York,NaN,86th And 87th St,0.40,40.621118,-74.026575,Dyker Heights,11209,NaN,NaN,NY,NaN,NaN,Lily Bloom Bay Ridge,Bakery,NaN
10318,5df97fa640ed0900087de054,190 Avenue B,Manhattan,NaN,New York,NaN,East 12th Street,0.16,40.727979,-73.979199,East Village,10009,NaN,NaN,NY,NaN,NaN,Ama Raw Bar,Seafood Restaurant,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10417,60464f6125275b1f8f0060d2,1382 Nostrand Ave,Brooklyn,NaN,New York,NaN,NaN,0.41,NaN,-73.949877,Erasmus,11226,NaN,NaN,NY,NaN,NaN,Mug&Cup,Café,NaN
10418,6048e6b13c09767941de32e7,155 Lafayette Ave,Staten_Island,NaN,Staten Island,NaN,NaN,0.42,NaN,-74.094771,Randall Manor,10301,NaN,NaN,NY,NaN,NaN,Casa Blanca,Spanish Restaurant,NaN
10420,604ae77621921d08a37d8cd4,825 Morris Park Ave,Bronx,NaN,New York,NaN,Matthews Ave,0.31,40.846332,-73.861653,Van Nest,10462,NaN,NaN,NY,NaN,NaN,La Catrina,Mexican Restaurant,NaN
10421,604bebdb59c1ce087b17a4a5,1761 Crosby Ave,Bronx,NaN,Bronx,NaN,NaN,0.45,NaN,-73.832030,Country Club,10461,NaN,NaN,NY,NaN,NaN,Taqueria Guerrero,Mexican Restaurant,NaN


In [17]:
# Run Restaurant Details: Rating

new_details_list = []
for index, venue_id in new_details_search_df['ID'].items():
    #print(venue_id)
    _details_json = foursquare_venue_details(My_FourSquare_dict, venue_id)
    if (_details_json['meta']['code'] == 429) | (_details_json['meta']['code'] == 430): 
        break 
    else:
        _series = parse_venue_details(_details_json)
        new_details_list.append(_series)

new_details_results_df = pd.DataFrame(new_details_list)
# print(_details_json['meta']['code'])

In [18]:
new_details_results_df 
# = new_venue_details_df[new_venue_details_df['ID'].notna()]

,ID,Venue_Name,Rating,Price,Created_At,Checkins,Users,Tips,Visits
0,4ba29ed3f964a520e90938e3,H.I.M. Ital Health Food Market,NaN,2.0,1268948691,NaN,NaN,4,NaN
1,3fd66200f964a520e0f01ee3,New Stadium Pizza,NaN,1.0,1071014400,NaN,NaN,3,NaN
2,4fcbfc92e4b088c92e7e97ad,Opera Cafe Lounge,8.4,3.0,1338768530,NaN,NaN,54,NaN
3,559d92a7498e66a57196c993,Caffe Bene,6.1,1.0,1436390055,NaN,NaN,7,NaN
4,5df97fa640ed0900087de054,Ama Raw Bar,8.5,3.0,1576632230,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...
79,60464f6125275b1f8f0060d2,Mug&Cup,NaN,1.0,1615220577,NaN,NaN,0,NaN
80,6048e6b13c09767941de32e7,Casa Blanca,NaN,2.0,1615390385,NaN,NaN,0,NaN
81,604ae77621921d08a37d8cd4,La Catrina,NaN,1.0,1615521654,NaN,NaN,0,NaN
82,604bebdb59c1ce087b17a4a5,Taqueria Guerrero,NaN,1.0,1615588315,NaN,NaN,0,NaN


In [19]:
new_details_results_df['ID'].isna().sum()

0

In [20]:
new_details_results_df['Rating'].isna().sum()

26

In [21]:
day

1

In [22]:
# Save to CSV:
new_details_results_df.to_csv(path_or_buf= home_path / f'FourSquare_Details_Day_{day}.csv')


In [23]:
# Previous # of Restaurant Details:
print("Previous # of Restaurant Details: {}"
      .format(len(all_details_df[all_details_df['Rating'].notna()]['ID'].unique())))


Previous # of Restaurant Details: 8110


In [24]:
# Replace unrated restaurants with a zero rating:
new_details_results_df['Rating'] = new_details_results_df['Rating'].fillna(value=0)
new_details_results_df

,ID,Venue_Name,Rating,Price,Created_At,Checkins,Users,Tips,Visits
0,4ba29ed3f964a520e90938e3,H.I.M. Ital Health Food Market,0.0,2.0,1268948691,NaN,NaN,4,NaN
1,3fd66200f964a520e0f01ee3,New Stadium Pizza,0.0,1.0,1071014400,NaN,NaN,3,NaN
2,4fcbfc92e4b088c92e7e97ad,Opera Cafe Lounge,8.4,3.0,1338768530,NaN,NaN,54,NaN
3,559d92a7498e66a57196c993,Caffe Bene,6.1,1.0,1436390055,NaN,NaN,7,NaN
4,5df97fa640ed0900087de054,Ama Raw Bar,8.5,3.0,1576632230,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...
79,60464f6125275b1f8f0060d2,Mug&Cup,0.0,1.0,1615220577,NaN,NaN,0,NaN
80,6048e6b13c09767941de32e7,Casa Blanca,0.0,2.0,1615390385,NaN,NaN,0,NaN
81,604ae77621921d08a37d8cd4,La Catrina,0.0,1.0,1615521654,NaN,NaN,0,NaN
82,604bebdb59c1ce087b17a4a5,Taqueria Guerrero,0.0,1.0,1615588315,NaN,NaN,0,NaN


In [25]:
# Join New Restaurant Details to All Venues Table

new_details_merge_df = (all_details_df.set_index('ID').combine_first(new_details_results_df.set_index('ID'))
                        .reset_index())

print("New # of Restaurant Details: {}"
      .format(len(new_details_merge_df[new_details_merge_df['Rating'].notna()]['ID'].unique())))

New # of Restaurant Details: 8190


In [26]:
new_details_merge_df[new_details_merge_df['Rating'].notna()]

,ID,Address,Borough,Checkins,City,Created_At,Cross_Street,Distance,Lat,Long,NYC_Neighborhood,Postal_Code,Price,Rating,State,Tips,Users,Venue_Name,Venue_Type,Visits
0,3fd66200f964a52006e91ee3,1274 Amsterdam Ave,Manhattan,NaN,New York,1.071014e+09,W 123rd St.,0.40,40.811241,-73.957869,Manhattanville,10027,2.0,7.8,NY,60.0,NaN,Max SoHa,Italian Restaurant,NaN
1,3fd66200f964a52044ea1ee3,1274 Amsterdam Ave,Manhattan,NaN,New York,1.071014e+09,W 123rd St.,0.39,40.811241,-73.957869,Morningside Heights,10027,2.0,8.7,NY,470.0,NaN,J.G. Melon,Italian Restaurant,NaN
2,3fd66200f964a5205fee1ee3,72 Bedford St,Manhattan,NaN,New York,1.071014e+09,Commerce,0.22,40.731493,-74.004825,West Village,10014,3.0,7.7,NY,17.0,NaN,Ponte Vecchio Restaurant,Brazilian Restaurant,NaN
3,3fd66200f964a5207bf11ee3,71 Irving Pl,Manhattan,NaN,New York,1.071014e+09,btwn E 18th & E 19th St,0.29,40.736716,-73.986817,Gramercy,10003,3.0,8.6,NY,72.0,NaN,F & J Pine Tavern,Café,NaN
4,3fd66200f964a52080e41ee3,172 Thompson St,Manhattan,NaN,New York,1.071014e+09,btwn Bleecker & Houston St,0.06,40.727743,-74.000267,Greenwich Village,10012,4.0,8.9,NY,122.0,NaN,Craft,Sushi Restaurant,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10420,604ae77621921d08a37d8cd4,825 Morris Park Ave,Bronx,NaN,New York,1.615522e+09,Matthews Ave,0.31,40.846332,-73.861653,Van Nest,10462,1.0,0.0,NY,0.0,NaN,La Catrina,Mexican Restaurant,NaN
10421,604bebdb59c1ce087b17a4a5,1761 Crosby Ave,Bronx,NaN,Bronx,1.615588e+09,NaN,0.45,NaN,-73.832030,Country Club,10461,1.0,0.0,NY,0.0,NaN,Taqueria Guerrero,Mexican Restaurant,NaN
10422,604bebdb59c1ce087b17a4a5,1761 Crosby Ave,Bronx,NaN,Bronx,1.615588e+09,NaN,0.28,NaN,-73.832030,Pelham Bay,10461,1.0,0.0,NY,0.0,NaN,Taqueria Guerrero,Mexican Restaurant,NaN
10423,604e11201ee66f712571074f,259-19 Union Tpke,Queens,NaN,Glen Oaks,1.615729e+09,NaN,0.23,NaN,-73.712715,Glen Oaks,11004,1.0,0.0,NY,0.0,NaN,Subway Sandwiches,Sandwich Place,NaN


In [27]:
new_details_merge_df

,ID,Address,Borough,Checkins,City,Created_At,Cross_Street,Distance,Lat,Long,NYC_Neighborhood,Postal_Code,Price,Rating,State,Tips,Users,Venue_Name,Venue_Type,Visits
0,3fd66200f964a52006e91ee3,1274 Amsterdam Ave,Manhattan,NaN,New York,1.071014e+09,W 123rd St.,0.40,40.811241,-73.957869,Manhattanville,10027,2.0,7.8,NY,60.0,NaN,Max SoHa,Italian Restaurant,NaN
1,3fd66200f964a52044ea1ee3,1274 Amsterdam Ave,Manhattan,NaN,New York,1.071014e+09,W 123rd St.,0.39,40.811241,-73.957869,Morningside Heights,10027,2.0,8.7,NY,470.0,NaN,J.G. Melon,Italian Restaurant,NaN
2,3fd66200f964a5205fee1ee3,72 Bedford St,Manhattan,NaN,New York,1.071014e+09,Commerce,0.22,40.731493,-74.004825,West Village,10014,3.0,7.7,NY,17.0,NaN,Ponte Vecchio Restaurant,Brazilian Restaurant,NaN
3,3fd66200f964a5207bf11ee3,71 Irving Pl,Manhattan,NaN,New York,1.071014e+09,btwn E 18th & E 19th St,0.29,40.736716,-73.986817,Gramercy,10003,3.0,8.6,NY,72.0,NaN,F & J Pine Tavern,Café,NaN
4,3fd66200f964a52080e41ee3,172 Thompson St,Manhattan,NaN,New York,1.071014e+09,btwn Bleecker & Houston St,0.06,40.727743,-74.000267,Greenwich Village,10012,4.0,8.9,NY,122.0,NaN,Craft,Sushi Restaurant,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10420,604ae77621921d08a37d8cd4,825 Morris Park Ave,Bronx,NaN,New York,1.615522e+09,Matthews Ave,0.31,40.846332,-73.861653,Van Nest,10462,1.0,0.0,NY,0.0,NaN,La Catrina,Mexican Restaurant,NaN
10421,604bebdb59c1ce087b17a4a5,1761 Crosby Ave,Bronx,NaN,Bronx,1.615588e+09,NaN,0.45,NaN,-73.832030,Country Club,10461,1.0,0.0,NY,0.0,NaN,Taqueria Guerrero,Mexican Restaurant,NaN
10422,604bebdb59c1ce087b17a4a5,1761 Crosby Ave,Bronx,NaN,Bronx,1.615588e+09,NaN,0.28,NaN,-73.832030,Pelham Bay,10461,1.0,0.0,NY,0.0,NaN,Taqueria Guerrero,Mexican Restaurant,NaN
10423,604e11201ee66f712571074f,259-19 Union Tpke,Queens,NaN,Glen Oaks,1.615729e+09,NaN,0.23,NaN,-73.712715,Glen Oaks,11004,1.0,0.0,NY,0.0,NaN,Subway Sandwiches,Sandwich Place,NaN


In [28]:
new_details_merge_df['ID'].isna().sum()

0

In [29]:
# new_details_merge_df[new_details_merge_df['ID'].notna()]

In [30]:
# Save to CSV
new_details_merge_df.to_csv(path_or_buf= home_path / 'FourSquare_Details_All_Restaurants.csv')

In [31]:
# Build FourSquare Neighborhood Quality Metric:

foursquare_pivot = pd.pivot_table(all_details_df[['Borough', 'NYC_Neighborhood', 'Venue_Name', 'Rating']],
                                  index=['Borough', 'NYC_Neighborhood'], columns='Venue_Name', values='Rating')

foursquare_pivot['Above_8point5'] = foursquare_pivot.apply(lambda x: x[x >= 8.5].count(), axis=1)
foursquare_pivot['Above_8'] = foursquare_pivot.apply(lambda x: x[x >= 8.0].count(), axis=1)
foursquare_pivot['Above_9'] = foursquare_pivot.apply(lambda x: x[x >= 9.0].count(), axis=1)

neighborhood_quality_df = foursquare_pivot[['Above_8point5','Above_8','Above_9']]


In [32]:
neighborhood_quality_df.sort_values('Above_8point5', ascending=False)  # 'Above_8point5','Above_8','Above_9'

Venue_Name                       Above_8point5  Above_8  Above_9
Borough       NYC_Neighborhood                                  
Manhattan     Noho                          33       48       14
              Greenwich Village             33       45       16
              Flatiron                      32       43       11
              Soho                          31       43       13
              Lower East Side               31       42       14
...                                        ...      ...      ...
Brooklyn      Borough Park                   0        3        0
Queens        Jamaica Estates                0        1        0
Brooklyn      Broadway Junction              0        0        0
              Brownsville                    0        0        0
Staten_Island Woodrow                        0        1        0

[305 rows x 3 columns]

In [38]:
# Save to CSV:
neighborhood_quality_df.to_csv(path_or_buf= home_path / 'FourSquare_Neighborhood_Quality_df.csv')


In [37]:
# neighborhood_quality_df.loc['Staten_Island','Todt Hill']  # Error
all_restaurants_df = pd.read_csv(home_path / 'FourSquare_All_Restaurants.csv', index_col=0)
all_restaurants_df[all_restaurants_df['NYC_Neighborhood'] == 'Todt Hill']

,ID,Venue_Name,Venue_Type,Lat,Long,Distance,Address,Cross_Street,Postal_Code,City,State,NYC_Neighborhood,Borough


In [30]:
# neighborhood_quality_df['Above_8'].value_counts()